In [1]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation, Bidirectional
from keras.utils import np_utils

import matplotlib.pyplot as plt

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
train = pd.read_csv('dataset/train_emoji.csv',header=None)
test = pd.read_csv('dataset/test_emoji.csv',header=None)

train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [3]:
import emoji


emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":grinning_face_with_big_eyes:",
                    "3": ":disappointed_face:",
                    "4": ":fork_and_knife:",
                    "5": ":hundred_points:",
                    "6": ":fire:",
                    "7": ":face_blowing_a_kiss:",
                    "8": ":chestnut:",
                    "9":":flexed_biceps:"
                   }


In [4]:
emoji.EMOJI_UNICODE

{u':raising_hands_dark_skin_tone:': u'\U0001f64c\U0001f3ff',
 u':Azerbaijan:': u'\U0001f1e6\U0001f1ff',
 u':man_gesturing_NO_medium-light_skin_tone:': u'\U0001f645\U0001f3fc\u200d\u2642\ufe0f',
 u':clapping_hands_medium_skin_tone:': u'\U0001f44f\U0001f3fd',
 u':woman_facepalming_light_skin_tone:': u'\U0001f926\U0001f3fb\u200d\u2640\ufe0f',
 u':man_in_lotus_position_light_skin_tone:': u'\U0001f9d8\U0001f3fb\u200d\u2642\ufe0f',
 u':person_gesturing_NO_medium-light_skin_tone:': u'\U0001f645\U0001f3fc',
 u':baby_medium-light_skin_tone:': u'\U0001f476\U0001f3fc',
 u':microscope:': u'\U0001f52c',
 u':person_walking_medium_skin_tone:': u'\U0001f6b6\U0001f3fd',
 u':women\u2019s_room:': u'\U0001f6ba',
 u':Cuba:': u'\U0001f1e8\U0001f1fa',
 u':pregnant_woman:': u'\U0001f930',
 u':person_mountain_biking_medium_skin_tone:': u'\U0001f6b5\U0001f3fd',
 u':left-facing_fist_medium_skin_tone:': u'\U0001f91b\U0001f3fd',
 u':backhand_index_pointing_down_medium-light_skin_tone:': u'\U0001f447\U0001f3fc',
 u

In [5]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

⚾
\u2764\uFE0F
😞
😃
💯
🍴
😘
🔥
💪
🌰


In [6]:
data  = train.values
print(data)
for i in range(5):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))
    

[['never talk to me again' 3 nan nan]
 ['I am proud of your achievements' 2 nan nan]
 ['It is the worst day in my life' 3 nan nan]
 ['Miss you so much' 0 nan ' [0]']
 ['food is life' 4 nan nan]
 ['I love you mum' 0 nan nan]
 ['Stop saying bullshit' 3 nan nan]
 ['congratulations on your acceptance' 2 nan nan]
 ['The assignment is too long ' 3 nan nan]
 ['I want to go play' 1 nan ' [3]']
 ['she did not answer my text ' 3 nan nan]
 ['Your stupidity has no limit' 3 nan nan]
 ['how many points did he score' 1 nan nan]
 ['my algorithm performs poorly' 3 nan nan]
 ['I got approved' 2 nan nan]
 ['Stop shouting at me' 3 nan nan]
 ['Sounds like a fun plan ha ha' 2 nan nan]
 ['no one likes him' 3 nan nan]
 ['the game just finished' 1 nan ' [2]']
 ['I will celebrate soon' 2 nan nan]
 ['So sad you are not coming' 3 nan nan]
 ['She is my dearest love' 0 nan ' [1]']
 ['Good job' 2 nan ' [4]']
 ['It was funny lol' 2 nan nan]
 ['candy is life ' 2 nan nan]
 ['The chicago cubs won again' 1 nan nan]
 ['I 

In [7]:
print(data.shape)

(132, 4)


In [8]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print(X_test.shape)

(56,)


In [9]:

for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()
    
for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()

/home/munishgrover/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/munishgrover/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
print(X_train.shape)

(132,)


In [11]:
Y_train = np_utils.to_categorical(Y_train)
        

In [12]:
Yt = np_utils.to_categorical(Y_test)

In [13]:
print(X_train[0],Y_train[0])

(['never', 'talk', 'to', 'me', 'again'], array([0., 0., 0., 1., 0.], dtype=float32))


In [14]:
embeddings_index = {}
f = open('glove.6B.50d.txt')

In [15]:
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float')
    embeddings_index[word] = coefs
f.close()

In [16]:
print(embeddings_index['food'].shape)
print(embeddings_index.get('food'))
print(X_train[0])

(50,)
[ 0.47222   -0.44545   -0.51833   -0.26818    0.44427   -0.25108
 -0.99282   -0.90198    1.8729     0.039081   0.14284    0.074878
  1.0543    -0.3203     1.0722     0.44323    0.0099484  0.15754
  0.51399   -0.77668    0.924      0.010958   0.58815    0.23078
 -0.34281   -0.88444   -0.31492    0.12661    1.1445     0.60775
  3.4344     0.63561   -0.13832    0.28045   -0.16181    0.77541
 -0.49888    0.4602     0.91799    0.29007    0.06884    0.59978
  0.53967   -0.061752   1.2975     0.92323   -0.80945    0.34932
  0.33934    0.25499  ]
['never', 'talk', 'to', 'me', 'again']


In [17]:
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        try:
            embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]   
        except:
            embedding_matrix_test[ix][ij] = np.zeros((50,))

In [18]:
print(embedding_matrix_train.shape,embedding_matrix_test.shape)
print( embedding_matrix_train[0])
print(embeddings_index.get('never'))

((132, 10, 50), (56, 10, 50))
[[ 0.095387  -0.16865   -0.11514   -0.51114    0.38331    0.22658
  -0.78504    0.67626   -0.66857    0.18847    0.19963    0.58351
  -0.86134   -0.39472    1.1571     0.51657    0.11706    0.0062629
  -0.2593    -0.33371   -0.47957    0.6211     0.66831   -0.058046
   0.81305   -2.341     -0.75437    0.2167     0.78012   -0.81362
   2.9368     0.13466   -0.38043   -0.59615   -0.093113  -0.2843
   0.28314    0.59791   -0.20751   -0.43841   -0.34187   -0.21166
  -0.082453   0.44007   -0.3365    -0.091078  -0.45859   -0.42103
  -0.53817    0.13738  ]
 [-0.034604   0.41087   -0.097368  -0.19548   -0.05855   -0.2581
  -0.85141    0.19582   -0.76829    0.37035   -0.63907    0.30593
  -0.38117    0.40505    0.90915    0.54359   -0.088861   0.17422
   0.33587   -0.36806    0.32399    1.056      0.68695    0.81261
   0.74166   -1.7618    -0.085259   0.34565    0.55371   -0.24026
   2.8321     0.80733   -0.59279   -0.52127   -0.60045   -0.39173
  -0.27123   -0.5200

In [19]:
model = Sequential()

model.add(LSTM(64, input_shape=(10,50), return_sequences=True,recurrent_dropout=0.1))
model.add(Dropout(0.5))
model.add(LSTM(64,input_shape=(10,50), return_sequences=False,recurrent_dropout=0.1))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/home/munishgrover/.local/lib/python2.7/site-packages/keras/layers/recurrent.py:2155: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [20]:
from keras.callbacks import ModelCheckpoint # save the best model, fight overfiitting
from keras.callbacks import EarlyStopping #save time


checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
earlystop = EarlyStopping(monitor='val_acc',patience=10)



#earlystop = EarlyStopping(patience=10)

hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 200, batch_size=32,shuffle=True,validation_split=0.1,callbacks=[checkpoint,earlystop])

Train on 118 samples, validate on 14 samples
Epoch 1/200
118/118 [==============================] - 0s 1ms/step - loss: 1.6132 - acc: 0.2119 - val_loss: 1.6044 - val_acc: 0.3571
Epoch 2/200
118/118 [==============================] - 0s 949us/step - loss: 1.5464 - acc: 0.3898 - val_loss: 1.6106 - val_acc: 0.2143
Epoch 3/200
118/118 [==============================] - 0s 1ms/step - loss: 1.5140 - acc: 0.3898 - val_loss: 1.6255 - val_acc: 0.1429
Epoch 4/200
118/118 [==============================] - 0s 1ms/step - loss: 1.4854 - acc: 0.3729 - val_loss: 1.6463 - val_acc: 0.2143
Epoch 5/200
118/118 [==============================] - 0s 1ms/step - loss: 1.4601 - acc: 0.3898 - val_loss: 1.6231 - val_acc: 0.3571
Epoch 6/200
118/118 [==============================] - 0s 1ms/step - loss: 1.4300 - acc: 0.4492 - val_loss: 1.6079 - val_acc: 0.2857
Epoch 7/200
118/118 [==============================] - 0s 1ms/step - loss: 1.3610 - acc: 0.4407 - val_loss: 1.5276 - val_acc: 0.3571
Epoch 8/200
118/118 [=

In [21]:
pred = model.predict_classes(embedding_matrix_test)

In [22]:
print(pred)
for i in range(X_test.shape[0]):
   print(X_test[i])
   print(emoji.emojize(emoji_dictionary.get((str)(pred[i]))))

[4 3 2 2 2 2 3 2 4 2 1 2 0 0 1 3 2 2 3 2 0 0 4 0 3 3 2 0 1 2 0 1 0 2 0 1 2
 4 4 2 1 0 0 1 2 2 2 2 3 1 3 0 3 2 2 3]
['I', 'want', 'to', 'eat']
🍴
['he', 'did', 'not', 'answer']
😞
['he', 'got', 'a', 'raise']
😃
['she', 'got', 'me', 'a', 'present']
😃
['ha', 'ha', 'ha', 'it', 'was', 'so', 'funny']
😃
['he', 'is', 'a', 'good', 'friend']
😃
['I', 'am', 'upset']
😞
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight']
😃
['where', 'is', 'the', 'food']
🍴
['Stop', 'making', 'this', 'joke', 'ha', 'ha', 'ha']
😃
['where', 'is', 'the', 'ball']
⚾
['work', 'is', 'hard']
😃
['This', 'girl', 'is', 'messing', 'with', 'me']
\u2764\uFE0F
['are', 'you', 'serious', 'ha', 'ha']
\u2764\uFE0F
['Let', 'us', 'go', 'play', 'baseball']
⚾
['This', 'stupid', 'grader', 'is', 'not', 'working']
😞
['work', 'is', 'horrible']
😃
['Congratulation', 'for', 'having', 'a', 'baby']
😃
['stop', 'messing', 'around']
😞
['any', 'suggestions', 'for', 'dinner']
😃
['I', 'love', 'taking', 'breaks']
\u2764\uFE0F
['you', 'brighten', 'my', 'd

In [23]:
t = np.array(test[1])

In [24]:
float(sum(pred==t))/embedding_matrix_test.shape[0]

0.6428571428571429

In [25]:
model.load_weights("best_model.h5")

In [26]:
pred = model.predict_classes(embedding_matrix_test)
t = np.array(test[1])
float(sum(pred==t))/embedding_matrix_test.shape[0]

0.5892857142857143